In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from torchinfo import summary
from models import XSwinFusion 
from pose_estimation import PoseDataNPZTorch


WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
x = torch.tensor([0, 1, 2, 3, 4, 5])

x = x[[2,3, 2,3,2 ,3 ,2 ,3]]
x


tensor([2, 3, 2, 3, 2, 3, 2, 3])

In [3]:
x = {"x"}

if "r" in x:
    print(x)


In [5]:
samples = 50_000
resize = (432, 768)
aspect_ratio = True
margin = 12
feature_dims = 64

fusion = XSwinFusion(feature_dims=feature_dims, resize=resize)

dataset = None
dataset = PoseDataNPZTorch(DATASET_NPZ_PATH, samples=samples, 
                           resize=resize, aspect_ratio=aspect_ratio, margin=margin)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


OSError: [Errno 24] Too many open files: '/Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz/scenes/2-14-16.npz'

In [4]:
points = []

for i, (s, t, c, d, m_info, p) in enumerate(dataloader):

    print(t.shape, c.shape, d.shape)

    rgb, pcd, transforms = fusion(t, c, m_info)


    assert 0


(70116,)
(50961,)
torch.Size([2, 50000, 3]) torch.Size([2, 3, 432, 768]) torch.Size([2, 432, 768])
torch.Size([2, 432, 768]) torch.Size([2]) torch.Size([2, 50000])
torch.Size([2, 432, 768, 64]) torch.Size([2, 64, 50000])
torch.Size([121077, 64])


AssertionError: 